# V3-NN (from WAV): Heart Sound Classification

This notebook trains a **Neural Network (Multi-Layer Perceptron)** to classify heart sounds by **loading raw `.wav` files directly** and applying wavelet transform in real-time.

Workflow:

1.  **Configuration**: Set up paths and parameters.
2.  **Feature Extraction**: Define a function to load a `.wav` file, apply DWT, and extract statistical features.
3.  **Data Loading**: Iterate through the original training datasets (`training-a`, `training-b`, etc.), process each audio file, and load corresponding labels.
4.  **Model Definition**: Define and build a sequential neural network with multiple layers.
5.  **Model Training & Evaluation**: Compile, train, and evaluate the model.

In [1]:
import os
import numpy as np
import pandas as pd
import librosa
import pywt
from scipy.stats import skew, kurtosis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# --- 1. Configuration ---
BASE_PROJECT_DIR = '../../' # Adjust path based on Docker container's file structure
TRAINING_SETS = [f'training-{letter}' for letter in ['a', 'b', 'c', 'd', 'e', 'f']]
WAVELET_FAMILY = 'db4'

2025-07-30 05:01:19.052526: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-30 05:01:19.055676: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 05:01:19.078893: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-30 05:01:19.107254: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753851679.132707   30250 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753851679.13

In [2]:
# --- 2. Feature Extraction from WAV ---
def extract_wavelet_features(audio_path):
    """Loads a WAV file, applies DWT, and extracts statistical features."""
    features = []
    try:
        y, sr = librosa.load(audio_path, sr=2000)
        
        # Apply DWT
        (cA, cD) = pywt.dwt(y, WAVELET_FAMILY)
        
        # Calculate statistical features for both coefficient arrays
        for coeffs in [cA, cD]:
            features.extend([
                np.mean(coeffs),
                np.std(coeffs),
                skew(coeffs),
                kurtosis(coeffs),
                np.sum(np.square(coeffs)) # Energy
            ])
        return np.array(features)
    except Exception as e:
        print(f"Error processing {os.path.basename(audio_path)}: {e}")
        return None

In [3]:
# --- 3. Data Loading and Feature Extraction ---
all_features = []
all_labels = []

for set_id in TRAINING_SETS:
    data_dir = os.path.join(BASE_PROJECT_DIR, set_id)
    labels_path = os.path.join(data_dir, 'REFERENCE.csv')
    
    if not os.path.exists(labels_path):
        print(f"Labels file not found for {set_id}, skipping.")
        continue

    labels_df = pd.read_csv(labels_path, header=None, names=['filename', 'label'])
    labels_df['label'] = labels_df['label'].apply(lambda x: 1 if x == 1 else 0)
    labels_dict = dict(zip(labels_df.filename, labels_df.label))

    print(f"Processing training set {set_id}...")
    for filename in os.listdir(data_dir):
        if filename.endswith('.wav'):
            file_id = os.path.splitext(filename)[0]
            if file_id in labels_dict:
                audio_path = os.path.join(data_dir, filename)
                features = extract_wavelet_features(audio_path)
                if features is not None:
                    all_features.append(features)
                    all_labels.append(labels_dict[file_id])

print("Feature extraction complete.")

# Convert to numpy arrays and scale
X = np.array(all_features)
y = np.array(all_labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Data prepared. Training samples: {X_train_scaled.shape[0]}, Test samples: {X_test_scaled.shape[0]}")

Processing training set training-a...
Processing training set training-b...
Processing training set training-c...
Processing training set training-d...
Processing training set training-e...
Processing training set training-f...
Feature extraction complete.
Data prepared. Training samples: 2592, Test samples: 648


In [4]:
# --- 4. Model Definition ---
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5), 
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.summary()

/usr/local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-07-30 05:01:42.608421: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,817 (11.00 KB)

 Trainable params: 2,817 (11.00 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# --- 5. Model Training & Evaluation ---
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Starting model training...")
history = model.fit(
    X_train_scaled, 
    y_train, 
    epochs=50, 
    batch_size=32, 
    validation_data=(X_test_scaled, y_test),
    verbose=1
)

print("--- Model Evaluation ---")
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Starting model training...
Epoch 1/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5560 - loss: 0.7036 - val_accuracy: 0.7932 - val_loss: 0.5409
Epoch 2/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8038 - loss: 0.5326 - val_accuracy: 0.7963 - val_loss: 0.4912
Epoch 3/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7834 - loss: 0.5259 - val_accuracy: 0.8040 - val_loss: 0.4600
Epoch 4/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7967 - loss: 0.4917 - val_accuracy: 0.8009 - val_loss: 0.4430
Epoch 5/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8049 - loss: 0.4736 - val_accuracy: 0.8117 - val_loss: 0.4310
Epoch 6/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7929 - loss: 0.4919 - val_accuracy: 0.8241 - val_loss: 0.4167
Epoch 7/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8035 - loss: 0.4687 - val_accuracy: 0.8210 - val_loss: 0.4072
Epoch 8/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8045 - loss: 0.4514 - val_a